<a href="https://colab.research.google.com/github/PCCraveiro/PCCraveiro-5IADT-Fase04-Grupo25/blob/main/Tech_Challenge_IADT_Fase_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Monta drive (ggogle drive)

from google.colab import drive
drive.mount('/content/drive')

print("✅ Drive montado")

Mounted at /content/drive
✅ Drive montado


In [2]:
# Instalações / Atualizações

!pip install -q --upgrade pip

# Core do projeto
!pip install -q ultralytics fer opencv-python tqdm
!pip install -q --upgrade ultralytics
!pip install -q fer tqdm

print("✅ Instalações / Atualizações")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 71.6 MB/s eta 0:00:00
✅ Instalações / Atualizações


In [3]:
import os
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from fer.fer import FER
from collections import Counter, defaultdict, deque
import json
from tqdm import tqdm
from huggingface_hub import hf_hub_download

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

print("✅ Bibliotecas importadas")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Bibliotecas importadas


In [4]:
# PARÂMETROS

# Pasta base do projeto dentro do seu Drive
DRIVE_BASE_DIR = "/content/drive/MyDrive/Colab Notebooks/Fase_4/"

os.makedirs(DRIVE_BASE_DIR, exist_ok=True)

# Paths de entrada/saída no Drive
VIDEO_PATH = os.path.join(DRIVE_BASE_DIR, "video.mp4")
OUTPUT_VIDEO = os.path.join(DRIVE_BASE_DIR, "video_analisado_v2.mp4")
REPORT_PATH = os.path.join(DRIVE_BASE_DIR, "relatorio_v2.json")

POSE_MODEL_PATH = "yolo11m-pose.pt"     # +13% precisão, ainda rápido

CONF_THRESHOLD = 0.35  # mais sensível
EMOTION_CONF_THRESHOLD = 0.4  # confiança mínima para aceitar emoção
IOU_THRESHOLD = 0.7
DEQUE_MAX_LEN = 12

pose_history = defaultdict(lambda: deque(maxlen=5))
speed_history = defaultdict(lambda: deque(maxlen=10))
action_history = defaultdict(lambda: deque(maxlen=5))

ANOMALY_SPEED_THRESHOLD = 35.0   # ajuste fino
ANOMALY_ZSCORE = 2.5

# --- ANOMALIAS ---
ANOMALY_SPEED_Z = 2.5
ANOMALY_MIN_SPEED = 20.0
ANOMALY_ACTION_WINDOW = 5
anomaly_count = 0

# Cores em BGR
COLORS = {
    "happy": (0, 255, 255),
    "sad": (255, 0, 0),
    "angry": (0, 0, 255),
    "surprise": (255, 255, 0),
    "fear": (255, 0, 255),
    "neutral": (200, 200, 200),
    "disgust": (0, 255, 0)
}

FACE_MODEL_PATH = hf_hub_download(
    repo_id="arnabdhar/YOLOv8-Face-Detection",
    filename="model.pt",
    local_dir="/content/models"
)

# Baixa tracker otimizado
!curl -O https://github.com/ultralytics/ultralytics/raw/main/ultralytics/cfg/trackers/bytetrack.yaml

print(f"✅ Face model baixado: {FACE_MODEL_PATH}")

model.pt:   0%|          | 0.00/6.25M [00:00<?, ?B/s]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
✅ Face model baixado: /content/models/model.pt


In [5]:
# DETECTOR DE CENA
class SceneDetector:
    """
    Detector simples de mudança de cena usando histograma de cor em HSV.
    Quando a correlação entre histograma atual e anterior fica abaixo
    do limiar, considera-se que há uma nova cena.
    """
    def __init__(self, threshold=0.7):
        self.threshold = threshold
        self.last_hist = None

    def detect_scene_change(self, frame):
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist([hsv], [0, 1], None, [50, 60], [0, 180, 0, 256])
        cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)

        is_new_scene = False
        if self.last_hist is not None:
            score = cv2.compareHist(self.last_hist, hist, cv2.HISTCMP_CORREL)
            if score < self.threshold:
                is_new_scene = True

        self.last_hist = hist
        return is_new_scene

In [6]:
# CLASSIFICADOR DE AÇÃO
class ActionClassifier:
    def __init__(self):
        self.history = {}
        self.velocities = {}

    def update(self, track_id, keypoints):
        if track_id not in self.history:
            self.history[track_id] = deque(maxlen=30)
            self.velocities[track_id] = deque(maxlen=10)

        self.history[track_id].append(keypoints)

        if len(self.history[track_id]) >= 2:
            prev_hip = self.history[track_id][-2][11]
            curr_hip = keypoints[11]
            if prev_hip[0] > 0 and curr_hip[0] > 0:
                speed = np.linalg.norm(curr_hip - prev_hip)
                self.velocities[track_id].append(speed)

    def classify(self, keypoints, track_id):
        kpts = keypoints
        if kpts.shape[0] < 17:
            return "Desconhecido"

        # 1. QUEDA
        shoulder_y = np.mean([kpts[5][1], kpts[6][1]])
        hip_y = np.mean([kpts[11][1], kpts[12][1]])
        ankle_y = np.mean([kpts[15][1], kpts[16][1]])

        if (shoulder_y > 0 and hip_y > 0 and ankle_y > 0 and
            shoulder_y > hip_y * 0.9 and hip_y > ankle_y * 0.8):
            return "DEITADO"

        # 2. MÃOS PARA CIMA
        nose_y = kpts[0][1]
        wrists_y = [kpts[9][1], kpts[10][1]]
        if nose_y > 0 and any(w > 0 and w < nose_y * 0.8 for w in wrists_y):
            return "MAOS PARA CIMA"

        # 3. SENTADO (anti-oclução + mesa)
        valid_kpts = kpts[kpts[:,0] > 0]
        if len(valid_kpts) > 8:
            # Critério 1: pernas dobradas OU tronco baixo
            knee_hip_dist = (np.linalg.norm(kpts[11] - kpts[13]) +
                            np.linalg.norm(kpts[12] - kpts[14]))
            shoulder_hip_dist = np.linalg.norm(kpts[5] - kpts[11]) + np.linalg.norm(kpts[6] - kpts[12])

            # Critério 2: braços na mesa (cotovelos próximos)
            elbow_dist = np.linalg.norm(kpts[7] - kpts[8])

            if knee_hip_dist < shoulder_hip_dist * 0.8 or elbow_dist < 80:
                return "SENTADO"

        # 4. CAMINHANDO/CORRENDO (CORRIGIDO)
        avg_speed = 0
        if track_id in self.velocities and len(self.velocities[track_id]) >= 5:
            avg_speed = np.mean(self.velocities[track_id])

            if avg_speed > 15:
                return "CORRENDO"
            elif avg_speed > 8:
                return "CAMINHANDO"

        return "EM PE"

In [7]:
# DASHBOARD OVERLAY
def draw_dashboard(img, frame_emotions, scene_num, scene_people_count, total_unique_people):
    """
    Desenha um pequeno dashboard sobre o frame com:
    - Número da cena
    - Pessoas na cena e total global
    - Barras de emoções detectadas no frame atual
    """
    overlay = img.copy()
    h, w, _ = img.shape
    dash_w = 280
    dash_h = 240
    x_start = w - dash_w - 20
    y_start = 20

    cv2.rectangle(overlay, (x_start, y_start), (x_start + dash_w, y_start + dash_h), (0, 0, 0), -1)
    alpha = 0.6
    img = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)

    cv2.putText(img, "DASHBOARD", (x_start + 80, y_start + 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    y_off = y_start + 55
    cv2.putText(img, f"Cena: {scene_num}", (x_start + 10, y_off),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
    y_off += 25
    cv2.putText(img, f"Pessoas (Cena): {scene_people_count}", (x_start + 10, y_off),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    y_off += 25
    cv2.putText(img, f"Pessoas (Total): {total_unique_people}", (x_start + 10, y_off),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1)

    y_off += 30
    cv2.putText(img, "EMOCOES (Ao Vivo)", (x_start + 10, y_off),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    y_off += 25

    total = sum(frame_emotions.values()) if frame_emotions else 1
    order = ["happy", "neutral", "surprise", "fear", "angry", "sad"]
    for emo in order:
        count = frame_emotions.get(emo, 0)
        color = COLORS.get(emo, (255, 255, 255))
        cv2.putText(img, f"{emo.capitalize()}", (x_start + 10, y_off),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        bar_w_max = 120
        bar_w = int((count / total) * bar_w_max) if total > 0 else 0
        if count > 0 and bar_w < 5:
            bar_w = 5
        cv2.rectangle(img, (x_start + 90, y_off - 10),
                      (x_start + 90 + bar_w, y_off), color, -1)
        cv2.putText(img, f"{count}", (x_start + 220, y_off),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        y_off += 20

    return img

In [8]:
# ASSOCIAÇÃO ROSTO-PESSOA
def get_face_for_person(kpts, face_boxes):
    """
    Encontra a melhor bounding box de rosto para um esqueleto (pessoa).
    Usa a média dos keypoints da cabeça (nariz, olhos, orelhas) como centro
    e escolhe o rosto mais próximo.
    """
    head_pts = kpts[0:5]
    valid_head = head_pts[head_pts[:, 0] > 0]

    if len(valid_head) == 0:
        return None

    head_center = np.mean(valid_head, axis=0)

    best_match = None
    min_dist = float('inf')

    for box in face_boxes:
        x1, y1, x2, y2 = box[:4]
        face_center = np.array([(x1 + x2) / 2, (y1 + y2) / 2])
        dist = np.linalg.norm(head_center - face_center)

        if dist < 100:  # limiar em pixels (ajuste conforme resolução)
            if dist < min_dist:
                min_dist = dist
                best_match = box

    return best_match

In [9]:
def detect_anomaly(track_id, action, action_classifier, action_history):
    is_speed_anomaly = False
    is_action_anomaly = False

    # --- 1) VELOCIDADE ANORMAL ---
    speeds = action_classifier.velocities.get(track_id, [])

    if len(speeds) >= 5:
        speeds_arr = np.array(speeds)
        mean = speeds_arr.mean()
        std = speeds_arr.std() + 1e-6
        z = (speeds_arr[-1] - mean) / std

        if speeds_arr[-1] > ANOMALY_MIN_SPEED or z > ANOMALY_SPEED_Z:
            is_speed_anomaly = True

    # --- 2) MUDANÇA BRUSCA DE AÇÃO ---
    action_history[track_id].append(action)

    if len(action_history[track_id]) >= 3:
        last_actions = list(action_history[track_id])
        # ação atual apareceu só uma vez na janela
        if last_actions.count(last_actions[-1]) == 1:
            is_action_anomaly = True

    return is_speed_anomaly or is_action_anomaly


In [10]:
# PROCESSAMENTO DO VÍDEO
def process_video():
    """
    Pipeline principal:
    - Carrega modelos de pose e rosto (YOLO) em GPU se disponível
    - Percorre o vídeo frame a frame
    - Detecta pessoas, ações, rostos e emoções
    - Detecta mudança de cena
    - Gera vídeo anotado e relatório JSON
    """
    # Dispositivo: prioriza CUDA (A100), caso não exista cai para CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    print("Loading models...")
    # Ultralytics carrega automaticamente o modelo local ou baixa da internet
    pose_model = YOLO(POSE_MODEL_PATH)  # modelo de pose
    try:
        face_model = YOLO(FACE_MODEL_PATH)  # modelo de face
        print("Face model loaded successfully.")
    except Exception as e:
        face_model = None
        print(f"WARNING: Face model failed to load: {e}")

    # Detector de emoções FER (usa internamente CNN + OpenCV)
    detector = FER(mtcnn=False)

    # Abre o vídeo (no Drive)
    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        raise FileNotFoundError(f"Não foi possível abrir o vídeo em: {VIDEO_PATH}")

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Writer para o vídeo anotado (no Drive)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (width, height))

    action_classifier = ActionClassifier()
    scene_detector = SceneDetector(threshold=0.7)

    # --- HISTÓRICO DE AÇÕES
    action_history = defaultdict(lambda: deque(maxlen=5))

    # Estatísticas finais (para o JSON)
    stats = {
        "emocoes": Counter(),
        "acoes": Counter(),
        "frames": 0,
        "total_cenas": 0,
        "pessoas_por_cena": {},  # {scene_id: count}
        "pessoas_global": set(),  # será convertido para lista na hora do dump
        "anomalias": 0,
        "anomalias_por_frame": []
    }

    emotion_buffer = {}
    last_emotions = {}

    # Controle de cenas e IDs locais
    scene_counter = 1
    scene_map = {}  # {global_track_id: local_scene_id}
    next_scene_person_id = 1
    current_scene_people = set()

    pbar = tqdm(total=total_frames, desc="Processing V2 Colab", ncols=100)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        stats["frames"] += 1
        annotated_frame = frame.copy()
        current_frame_emotions = Counter()

        # 1) Detecção de mudança de cena
        if scene_detector.detect_scene_change(frame):
            scene_counter += 1
            scene_map = {}
            next_scene_person_id = 1
            current_scene_people = set()
            action_classifier.history = {}
            print(f"Frame {stats['frames']}: Nova cena detectada ({scene_counter})")

        # 2) Detecção de rostos (opcional)
        face_boxes = []
        if face_model:
            face_results = face_model(frame, verbose=False, conf=0.4, device=device, half=True)
            if face_results and face_results[0].boxes is not None:
                face_boxes = face_results[0].boxes.xyxy.detach().cpu().numpy()

        # 3) Pose + tracking
        pose_results = pose_model.track(
            frame,
            persist=True,
            verbose=False,
            device=device,
            conf=CONF_THRESHOLD,
            iou=IOU_THRESHOLD,
            classes=[0], # só pessoas
            half=True  # FP16
        )

        # FILTRA POSES RUINS (robôs, objetos)
        if pose_results and pose_results[0].keypoints is not None:

            kpts_all = pose_results[0].keypoints.xy.detach().cpu().numpy()

            if pose_results[0].keypoints.conf is not None:
                kpts_conf = pose_results[0].keypoints.conf.detach().cpu().numpy()
            else:
                kpts_conf = np.ones((kpts_all.shape[0], kpts_all.shape[1]))

            ids = pose_results[0].boxes.id
            if ids is not None:
                ids = ids.int().detach().cpu().tolist()
            else:
                ids = [-1] * len(kpts_all)

            valid_indices = []

            for i, (kpts, confs) in enumerate(zip(kpts_all, kpts_conf)):
                num_good_kpts = np.sum(
                    (kpts[:, 0] > 0) &
                    (kpts[:, 1] > 0) &
                    (confs > 0.3)
                )
                if num_good_kpts >= 10:
                    valid_indices.append(i)

            # aplica filtro UMA ÚNICA VEZ
            if valid_indices:
                kpts_all = kpts_all[valid_indices]
                ids = [ids[i] if i < len(ids) else -1 for i in valid_indices]
            else:
                kpts_all = []
                ids = []

        if pose_results and pose_results[0].keypoints is not None:
            kpts_all = pose_results[0].keypoints.xy.detach().cpu().numpy()
            ids = pose_results[0].boxes.id
            if ids is not None:
                ids = ids.int().detach().cpu().tolist()
            else:
                ids = [-1] * len(kpts_all)

            for kpts, track_id in zip(kpts_all, ids):
                # Mapeamento de ID global -> ID local na cena
                if track_id != -1:
                    stats["pessoas_global"].add(int(track_id))
                    if track_id not in scene_map:
                        scene_map[track_id] = next_scene_person_id
                        next_scene_person_id += 1

                    local_id = scene_map[track_id]
                    current_scene_people.add(local_id)
                else:
                    local_id = "?"

                # Atualiza classificador de ação
                action_classifier.update(track_id, kpts)
                action = action_classifier.classify(kpts, track_id)
                stats["acoes"][action] += 1

                # --- DETECÇÃO DE ANOMALIA ---
                is_anomaly = False
                if track_id != -1:
                    is_anomaly = detect_anomaly(
                        track_id,
                        action,
                        action_classifier,
                        action_history
                    )

                    if is_anomaly:
                        stats["anomalias"] += 1
                        stats["anomalias_por_frame"].append({
                            "frame": stats["frames"],
                            "track_id": int(track_id),
                            "action": action,
                            "speed": float(
                                action_classifier.velocities[track_id][-1]
                                if action_classifier.velocities.get(track_id)
                                else 0
                            )
                        })

                # Desenha esqueleto
                skeleton = [
                    (5, 7), (7, 9), (6, 8), (8, 10),
                    (5, 6), (5, 11), (6, 12),
                    (11, 12), (11, 13), (13, 15),
                    (12, 14), (14, 16)
                ]
                for p1, p2 in skeleton:
                    if kpts[p1][0] > 0 and kpts[p2][0] > 0:
                        pt1 = (int(kpts[p1][0]), int(kpts[p1][1]))
                        pt2 = (int(kpts[p2][0]), int(kpts[p2][1]))
                        cv2.line(annotated_frame, pt1, pt2, (0, 255, 0), 2)

                # Emoções
                emotion_label = last_emotions.get(track_id, "neutral")

                # Tenta associar um rosto detectado a essa pessoa
                matched_face = get_face_for_person(kpts, face_boxes)
                face_img = None

                if matched_face is not None:
                    x1, y1, x2, y2 = matched_face[:4].astype(int)
                    pad = 10
                    x1 = max(0, x1 - pad)
                    y1 = max(0, y1 - pad)
                    x2 = min(width, x2 + pad)
                    y2 = min(height, y2 + pad)
                    if x2 > x1 and y2 > y1:
                        face_img = frame[y1:y2, x1:x2]

                # fallback: recorta região da cabeça a partir dos keypoints
                elif np.any(kpts[0:5, 0] > 0):
                    face_pts = kpts[0:5]
                    valid_face_pts = face_pts[face_pts[:, 0] > 0]
                    if len(valid_face_pts) > 0:
                        fx1, fy1 = np.min(valid_face_pts, axis=0).astype(int)
                        fx2, fy2 = np.max(valid_face_pts, axis=0).astype(int)
                        pad = 30
                        fx1 = max(0, fx1 - pad)
                        fy1 = max(0, fy1 - pad)
                        fx2 = min(width, fx2 + pad)
                        fy2 = min(height, fy2 + pad)
                        if fx2 > fx1 and fy2 > fy1:
                            face_img = frame[fy1:fy2, fx1:fx2]

                # Inferência de emoção se tivermos uma face recortada válida
                if face_img is not None and face_img.size > 0 and face_img.shape[0] > 20 and face_img.shape[1] > 20:
                    try:
                        emotions = detector.detect_emotions(face_img)
                        if emotions:
                            top_emotion, score = max(
                                emotions[0]["emotions"].items(),
                                key=lambda x: x[1]
                            )
                            if score > EMOTION_CONF_THRESHOLD:
                                if track_id not in emotion_buffer:
                                    emotion_buffer[track_id] = deque(maxlen=DEQUE_MAX_LEN)
                                emotion_buffer[track_id].append(top_emotion)
                                most_common = Counter(emotion_buffer[track_id]).most_common(1)[0][0]
                                emotion_label = most_common
                                last_emotions[track_id] = emotion_label
                    except Exception:
                        pass

                stats["emocoes"][emotion_label] += 1
                current_frame_emotions[emotion_label] += 1

                # Desenha label da pessoa
                if np.any(kpts[:, 0] > 0):
                    valid_x = kpts[kpts[:, 0] > 0, 0]
                    valid_y = kpts[kpts[:, 0] > 0, 1]
                    x1, y1 = int(np.min(valid_x)), int(np.min(valid_y))

                    color = COLORS.get(emotion_label, (0, 255, 0))

                    # Calcula proximidade com outras pessoas
                    # Label melhorado com contexto grupo
                    nearby_count = len(current_scene_people) - 1  # total na cena menos eu
                    label_text = f"Pessoa {local_id} | {action}"
                    if nearby_count > 1:
                        label_text += f" | Grupo:{nearby_count}"

                    emo_label = f"{emotion_label.upper()}"

                    (tw, th), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                    cv2.rectangle(annotated_frame, (x1, y1-50), (x1 + max(220, tw), y1), color, -1)
                    cv2.putText(annotated_frame, label_text, (x1, y1-25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
                    cv2.putText(annotated_frame, emo_label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

                    if is_anomaly:
                        cv2.putText(
                            annotated_frame,
                            "ANOMALIA",
                            (x1, y1 - 70),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.7,
                            (0, 0, 255),
                            3
                        )

        # Atualiza estatísticas por cena
        stats["total_cenas"] = scene_counter
        stats["pessoas_por_cena"][f"Cena {scene_counter}"] = len(current_scene_people)

        annotated_frame = draw_dashboard(
            annotated_frame,
            current_frame_emotions,
            scene_counter,
            len(current_scene_people),
            len(stats["pessoas_global"])
        )

        writer.write(annotated_frame)
        pbar.update(1)

    cap.release()
    writer.release()
    pbar.close()

    # Converte set para lista para serializar em JSON
    stats["pessoas_global"] = list(stats["pessoas_global"])

    # Salva relatório no Drive
    with open(REPORT_PATH, "w") as f:
        json.dump(stats, f, indent=4)

    print(f"Processamento concluído.")
    print(f"Vídeo anotado salvo em: {OUTPUT_VIDEO}")
    print(f"Relatório salvo em: {REPORT_PATH}")


# Execução
process_video()


Using device: cuda
Loading models...
Face model loaded successfully.


Processing V2 Colab:   0%|                                                 | 0/3326 [00:00<?, ?it/s]

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 122ms
Prepared 1 package in 20ms
Installed 1 package in 4ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 0.6s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect



Processing V2 Colab:   5%|██                                     | 181/3326 [00:21<03:19, 15.75it/s]

Frame 181: Nova cena detectada (2)


...

Frame 3121: Nova cena detectada (18)


Processing V2 Colab:  99%|█████████████████████████████████████▋| 3301/3326 [03:17<00:01, 14.58it/s]

Frame 3301: Nova cena detectada (19)


Processing V2 Colab: 100%|██████████████████████████████████████| 3326/3326 [03:18<00:00, 16.74it/s]


Processamento concluído.
Vídeo anotado salvo em: /content/drive/MyDrive/Colab Notebooks/Fase_4/video_analisado_v2.mp4
Relatório salvo em: /content/drive/MyDrive/Colab Notebooks/Fase_4/relatorio_v2.json
